# Building AI-powered search engine in PostgreSQL and pgvector

In [1]:
import os
from dotenv import load_dotenv
from langchain.vectorstores.pgvector import PGVector
import psycopg2
import numpy as np
from PIL import Image
import pandas as pd
from fashion_clip.fashion_clip import FashionCLIP


bin C:\Users\rusla\.conda\envs\textgen\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll


## Loading Data

Let's now FashionCLIP first. It's going to take a couple of minutes to download the model from the HuggingFace Hub.

In [2]:
#%%capture
fclip = FashionCLIP('fashion-clip')

In [3]:
#to get the current working directory
directory = os.getcwd()
# Join various path components
path=os.path.join(directory, "data_for_fashion_clip", "subset_data.csv")

In [4]:
subset = pd.read_csv(path)

In [5]:
subset.head(3)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,176754003,176754,2 Row Braided Headband (1),74,Hair/alice band,Accessories,1010016,Solid,17,Yellowish Brown,...,Hair Accessories,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories,Two-strand hairband with braids in imitation s...
2,189634031,189634,Long Leg Leggings,273,Leggings/Tights,Garment Lower body,1010016,Solid,93,Dark Green,...,Basic 1,D,Divided,2,Divided,51,Divided Basics,1002,Jersey Basic,Leggings in stretch jersey with an elasticated...


In [6]:
subset.to_csv("subset_data.csv", index=False)
f"There are {len(subset)} elements in the dataset"

'There are 3104 elements in the dataset'

##  Task 1: Creating a products table with vector data type and ingesting the data using PGVector


In [7]:
# Assuming you have already obtained the image and text embeddings
images = ["data_for_fashion_clip/" + str(k) + ".jpg" for k in subset["article_id"].tolist()]
texts = subset["detail_desc"].tolist()

# Create image embeddings and text embeddings
image_embeddings = fclip.encode_images(images, batch_size=32)
text_embeddings = fclip.encode_text(texts, batch_size=32)

# Normalize the embeddings to unit norm
image_embeddings = image_embeddings / np.linalg.norm(image_embeddings, ord=2, axis=-1, keepdims=True)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, ord=2, axis=-1, keepdims=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [02:49<00:00,  1.75s/it]


Map:   0%|          | 0/3104 [00:00<?, ? examples/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [00:05<00:00, 16.78it/s]


# Utils 

In [9]:
def test_connection(conn):
    # Create a cursor object
    cursor = conn.cursor()
    # Execute a test query
    cursor.execute("SELECT version();")
    # Fetch the result
    result = cursor.fetchone()
    print("Connection successful!")
    print("PostgreSQL version:", result[0])
    # Close the cursor and connection
    cursor.close()
    #conn.close()

## Task 2 : Connection to the Server

In [10]:
import psycopg2
from pgvector.psycopg2 import register_vector

# Establish connection to the PostgreSQL Docker container
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    user="testuser",
    password="testpwd",
    database="vectordb"
)
test_connection(conn)
# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define the SQL query to drop the table if it exists
drop_table_query = "DROP TABLE IF EXISTS products;"

# Execute the SQL query to drop the table
cursor.execute(drop_table_query)

# Commit the changes to the database
conn.commit()

# Define the SQL query to create the table
create_table_query = """\
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    image_embedding vector(512), -- Specify the shape of the vector
    text_embedding vector(512) -- Specify the shape of the vector
);
"""

# Execute the SQL query to create the table
cursor.execute(create_table_query)

# Commit the changes to the database
conn.commit()

# Define the SQL query to insert the data into the table
insert_data_query = "INSERT INTO products (image_embedding, text_embedding) VALUES (%s, %s)"

# Iterate through the embeddings and insert them into the table
for image_embedding, text_embedding in zip(image_embeddings[:100], text_embeddings[:100]):
    try:
        cursor.execute(insert_data_query, (image_embedding.tolist(), text_embedding.tolist()))
        # Commit the changes to the database
        conn.commit()
    except Exception as e:
        print(f"Error inserting data: {e}")
        # Rollback the transaction in case of an error
        conn.rollback()

Connection successful!
PostgreSQL version: PostgreSQL 15.4 (Debian 15.4-2.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


In [11]:
# Create an index for similarity search using L2 distance
create_index_query = "CREATE INDEX ON products USING ivfflat (text_embedding vector_l2_ops) WITH (lists = 100);"
cursor.execute(create_index_query)

# Commit the changes to the database
conn.commit()

# Close the cursor
cursor.close()

# Vacuum and analyze the table for optimal performance
vacuum_analyze_query = "VACUUM ANALYZE products;"
# Execute the VACUUM ANALYZE query outside the transaction block
conn.autocommit = True
cursor = conn.cursor()
cursor.execute(vacuum_analyze_query)

# Close the cursor and connection
cursor.close()
conn.close()

In [19]:
import numpy as np
import psycopg2

# Define the embedding function
def my_embedding_function(text):
    # Generate text embeddings using fclip.encode_text
    text_embeddings = fclip.encode_text([text], batch_size=1)
    # Normalize the embeddings to unit norm
    text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, ord=2, axis=-1, keepdims=True)
    return text_embeddings[0]  # Return the single text embedding

data = my_embedding_function("Blue jacket")
data_list = data.tolist()  # Convert numpy.ndarray to list

# Establish connection to the PostgreSQL Docker container
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    user="testuser",
    password="testpwd",
    database="vectordb"
)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 24.33it/s]


In [20]:
# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Get the column names in the products table
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'products';")
column_names = [row[0] for row in cursor.fetchall()]
print("Column Names:", column_names)

Column Names: ['id', 'image_embedding', 'text_embedding']


In [21]:
# Perform similarity search query
cursor.execute("""
    -- SELECT id, description, text_embedding
    SELECT id, text_embedding
    FROM products
    ORDER BY text_embedding <-> %s::vector
    LIMIT 2;
""", (data_list,))

# Fetch the results
results = cursor.fetchall()

In [22]:
# Process and display the results
for result in results:
    product_id = result[0]
    
    print("Product Item Id:", product_id)
    print("Description:", result[1])
   # print("Embedding:", result[2])
    print("----------------------")

# Close the cursor and connection
cursor.close()
conn.close()

Product Item Id: 48
Description: [0.023410298,-0.013177148,-0.015617458,0.009955669,-0.01777295,0.030018194,-0.010092812,0.064039744,-0.0076911673,0.017779142,-0.04886508,0.020496665,-0.010309409,-0.02337357,-0.06787995,-0.014375856,-0.040318474,0.022590628,0.004351568,0.00746218,0.008021529,-0.0040085693,0.01916376,-0.00507574,0.013201847,0.023242235,-0.0047811167,-0.026150428,-0.013631476,-0.02263303,-0.0066080065,0.019676488,-0.05809817,0.031445757,0.02649331,-0.036014147,0.053888433,0.06373215,-0.014435454,0.05046232,-0.028136535,-0.021974277,0.038241096,0.013659625,-0.02442254,0.050292015,-0.02547141,0.008108971,0.007807562,-0.02977,-0.055766474,0.034117892,0.028400697,0.026897002,0.09132406,0.043552235,-0.017686795,0.018791791,-0.023812372,-0.0638997,-0.009898719,0.0078092995,0.0046702824,-0.013159851,0.044242352,0.016539695,-0.019756798,-0.0130028585,0.032416962,0.031888053,-0.4391654,-0.023254678,0.0066879787,-0.0005796678,-0.029091904,-0.056399718,-0.022650972,0.026844967,0.02